In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp/preprocessed.csv')
df.head()

,News,Category,Cleaned_News,Stemmed
0,"\n﻿काठमाण्डौ, ६ असार । नेपाल ललितकला प्रज्ञा प...",politics,काठमाण्डौ ६ असार नेपाल ललितकला प्रज्ञा प्रतिष...,काठमाण्डौ ६ असार नेपाल ललितकला प्रज्ञा प्रतिष्...
1,नेपालको आशा जीवितैकप्तान पारस खड्काले ब्याट र ...,sport,नेपालको आशा जीवितैकप्तान पारस खड्काले ब्याट र ...,नेपाल आशा जीवितैकप्तान पारस खड्का ब्याट र शक्त...
2,"\n﻿राजविराज, २६ फागुन । नेकपा एमालेलाई औद्योगि...",politics,राजविराज २६ फागुन नेकपा एमालेलाई औद्योगिक क्ष...,राजविराज २६ फागुन नेकपा एमाले औद्योगिक क्षेत्र...
3,"सामाजिक कार्यका लागि सुन्दरी""\nसौन्दर्य प्रतिय...",entertainment,सामाजिक कार्यका लागि सुन्दरी\nसौन्दर्य प्रतियो...,सामाजिक कार्य लागि सुन्दरी सौन्दर्य प्रतियोगित...
4,"खराब प्रदर्शनपछि प्रशिक्षण पिच""महिला राष्ट्रिय...",sport,खराब प्रदर्शनपछि प्रशिक्षण पिचमहिला राष्ट्रिय ...,खराब प्रदर्शन प्रशिक्षण पिचमहिला राष्ट्रिय यू ...


In [ ]:
df['Category'].unique()

array(['politics', 'sport', 'entertainment', 'tech', 'business'],
      dtype=object)

**Tokenization and Vectorization**

In [ ]:
max_len = 500  # Max length of sequence
vocab_size = 50000  # Vocabulary size
embedding_dim = 350

In [ ]:
# Tokenization and vectorization
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Stemmed'])
word_index = tokenizer.word_index
print("Number of unique tokens:", len(word_index))

sequences = tokenizer.texts_to_sequences(df['Stemmed'])
max_length = max(len(seq) for seq in sequences)
print('max_length: ', max_length)
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

Number of unique tokens: 70679
max_length:  3288


**Save Tokenizer state for future use.**

In [ ]:
import io
import json

tokenizer_json = tokenizer.to_json()
with io.open('/content/drive/MyDrive/Colab Notebooks/nlp/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/nlp/tokenizer.json') as f:
#     data = json.load(f)
#     tokenizer = tokenizer_from_json(data)

In [ ]:
# encoding categories suitable for model training
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Category'])

In [ ]:
#split whole dataset into train, val, and test datasets
X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

**Model prepare and Train.**

In [ ]:
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

In [ ]:
vocab_size, embedding_dim, max_len

(50000, 350, 500)

In [ ]:
from keras.layers import SimpleRNN

# Model
model_simple_rnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    SimpleRNN(64),  # Simple RNN layer
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model_simple_rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model_simple_rnn.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 350)          17500000  
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                26560     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 17528805 (66.87 MB)
Trainable params: 17528805 (66.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Training
model_simple_rnn.fit(X_train, y_train, validation_data=(X_val, y_val),
                     epochs=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 69s 671ms/step - loss: 1.5630 - accuracy: 0.2738 - val_loss: 1.5041 - val_accuracy: 0.3392
Epoch 2/10
100/100 [==============================] - 67s 675ms/step - loss: 0.8839 - accuracy: 0.7464 - val_loss: 1.0979 - val_accuracy: 0.5683
Epoch 3/10
100/100 [==============================] - 65s 652ms/step - loss: 0.1059 - accuracy: 0.9846 - val_loss: 1.1793 - val_accuracy: 0.5742
Epoch 4/10
100/100 [==============================] - 75s 756ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 1.2965 - val_accuracy: 0.5698
Epoch 5/10
100/100 [==============================] - 68s 678ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 1.3241 - val_accuracy: 0.5742
Epoch 6/10
100/100 [==============================] - 67s 675ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.3577 - val_accuracy: 0.5756
Epoch 7/10
100/100 [==============================] - 87s 872ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.3885 - val_ac

In [ ]:
from keras.layers import GRU

# Model
model_gru = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GRU(64),  # GRU layer
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model_gru.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model_gru.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 350)          17500000  
                                                                 
 gru (GRU)                   (None, 64)                79872     
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 17582117 (67.07 MB)
Trainable params: 17582117 (67.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Training
model_gru.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 115s 1s/step - loss: 1.2570 - accuracy: 0.4660 - val_loss: 0.7565 - val_accuracy: 0.7327
Epoch 2/10
100/100 [==============================] - 111s 1s/step - loss: 0.3269 - accuracy: 0.8949 - val_loss: 0.3212 - val_accuracy: 0.9104
Epoch 3/10
100/100 [==============================] - 116s 1s/step - loss: 0.0804 - accuracy: 0.9786 - val_loss: 0.2914 - val_accuracy: 0.9163
Epoch 4/10
100/100 [==============================] - 113s 1s/step - loss: 0.0202 - accuracy: 0.9965 - val_loss: 0.2747 - val_accuracy: 0.9280
Epoch 5/10
100/100 [==============================] - 108s 1s/step - loss: 0.0065 - accuracy: 0.9994 - val_loss: 0.2729 - val_accuracy: 0.9339
Epoch 6/10
100/100 [==============================] - 114s 1s/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.2893 - val_accuracy: 0.9369
Epoch 7/10
100/100 [==============================] - 111s 1s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2949 - val_accuracy: 0.9369

In [ ]:
# LSTM Model
model_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model_lstm.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 350)          17500000  
                                                                 
 lstm (LSTM)                 (None, 64)                106240    
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 17608485 (67.17 MB)
Trainable params: 17608485 (67.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Training
model_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=1)

Epoch 1/10
100/100 [==============================] - 123s 1s/step - loss: 1.1661 - accuracy: 0.4997 - val_loss: 1.0019 - val_accuracy: 0.5492
Epoch 2/10
100/100 [==============================] - 119s 1s/step - loss: 0.5852 - accuracy: 0.7631 - val_loss: 0.4327 - val_accuracy: 0.8590
Epoch 3/10
100/100 [==============================] - 122s 1s/step - loss: 0.1593 - accuracy: 0.9534 - val_loss: 0.3276 - val_accuracy: 0.8943
Epoch 4/10
100/100 [==============================] - 119s 1s/step - loss: 0.0385 - accuracy: 0.9880 - val_loss: 0.3144 - val_accuracy: 0.9090
Epoch 5/10
100/100 [==============================] - 120s 1s/step - loss: 0.0110 - accuracy: 0.9987 - val_loss: 0.3139 - val_accuracy: 0.9134
Epoch 6/10
100/100 [==============================] - 118s 1s/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.3078 - val_accuracy: 0.9310
Epoch 7/10
100/100 [==============================] - 120s 1s/step - loss: 0.0039 - accuracy: 0.9994 - val_loss: 0.2950 - val_accuracy: 0.9325

**Accuracy Test and Classification Report**

In [ ]:
# Evaluate Simple RNN Model
_, accuracy_simple_rnn = model_simple_rnn.evaluate(X_test, y_test)
print("Simple RNN Test Accuracy:", accuracy_simple_rnn)

# Evaluate LSTM Model
_, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print("LSTM Test Accuracy:", accuracy_lstm)

# Evaluate GRU Model
_, accuracy_gru = model_gru.evaluate(X_test, y_test)
print("GRU Test Accuracy:", accuracy_gru)

22/22 [==============================] - 2s 78ms/step - loss: 1.2638 - accuracy: 0.6094
Simple RNN Test Accuracy: 0.6093979477882385
22/22 [==============================] - 5s 208ms/step - loss: 0.2939 - accuracy: 0.9457
LSTM Test Accuracy: 0.9456681609153748
22/22 [==============================] - 3s 138ms/step - loss: 0.3306 - accuracy: 0.9325
GRU Test Accuracy: 0.9324522614479065


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Make predictions
y_pred = model_simple_rnn.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)


categories = ['business', 'entertaintment', 'politics', 'sport', 'tech']
clr = classification_report(y_test, y_pred_labels, target_names=categories)
print('Simple RNN')
print(clr)

22/22 [==============================] - 1s 50ms/step
Simple RNN
                precision    recall  f1-score   support

      business       0.46      0.47      0.47       158
entertaintment       0.54      0.56      0.55       140
      politics       0.62      0.38      0.47        68
         sport       0.90      0.96      0.93       155
          tech       0.53      0.54      0.54       160

      accuracy                           0.61       681
     macro avg       0.61      0.58      0.59       681
  weighted avg       0.61      0.61      0.61       681



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Make predictions
y_pred = model_lstm.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)


categories = ['business', 'entertaintment', 'politics', 'sport', 'tech']
clr = classification_report(y_test, y_pred_labels, target_names=categories)
print('LSTM')
print(clr)

22/22 [==============================] - 4s 161ms/step
LSTM
                precision    recall  f1-score   support

      business       0.94      0.91      0.92       158
entertaintment       0.93      0.98      0.95       140
      politics       1.00      0.90      0.95        68
         sport       0.99      0.98      0.99       155
          tech       0.90      0.94      0.92       160

      accuracy                           0.95       681
     macro avg       0.95      0.94      0.95       681
  weighted avg       0.95      0.95      0.95       681



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Make predictions
y_pred = model_gru.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)


categories = ['business', 'entertaintment', 'politics', 'sport', 'tech']
clr = classification_report(y_test, y_pred_labels, target_names=categories)
print('GRU')
print(clr)

22/22 [==============================] - 6s 226ms/step
GRU
                precision    recall  f1-score   support

      business       0.95      0.89      0.92       158
entertaintment       0.90      0.98      0.94       140
      politics       1.00      0.81      0.89        68
         sport       0.97      0.97      0.97       155
          tech       0.89      0.94      0.92       160

      accuracy                           0.93       681
     macro avg       0.94      0.92      0.93       681
  weighted avg       0.94      0.93      0.93       681



In [18]:
#save models
# model_simple_rnn.save('/content/drive/MyDrive/Colab Notebooks/nlp/model_rnn.keras')
# model_lstm.save('/content/drive/MyDrive/Colab Notebooks/nlp/model_lstm.keras')
# model_gru.save('/content/drive/MyDrive/Colab Notebooks/nlp/model_gru.keras')

**Predict live text taken from nepali news website.**

In [3]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
import string
import json
import numpy as np
import re
import snowballstemmer

In [5]:
# Remove non-Nepali characters
def clean_text(text):
    cleaned_text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    return cleaned_text.strip()

#split sentences and remove punctuations
def sentence_tokenize(text):
    sentences = text.strip().split(u"।")
    sentences = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
    return sentences

#split words and remove trailing punctuations
def word_tokenize(sentence, new_punctuation=[]):
    punctuations = ['।', ',', ';', '?', '!', '—', '-', '.']
    if new_punctuation:
        punctuations = set(punctuations + new_punctuation)

    for punct in punctuations:
        sentence = ''.join(sentence.split(punct))

    return sentence

In [6]:
#clean sentence removing punctuations and non-nepali characters
def tokenize_text(text):
    sentences = sentence_tokenize(text)
    # print(sentences)
    cleaned_sentence_list = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        cleaned_sentence_list.append(words)

    cleaned_sentence = ''.join(cleaned_sentence_list)

    return cleaned_sentence


#stemming for nepali text
def stem_text(text):
    tokens = text.split()
    stemmer = snowballstemmer.NepaliStemmer()
    stemmed = stemmer.stemWords(tokens)

    return ' '.join(stemmed)

In [8]:
#news prediction
def predict_category(model, tokenizer, max_len, news_list):
    news = []
    for text in news_list:
        cleaned = tokenize_text(text)
        stemmed = stem_text(cleaned)
        news.append(stemmed)

    sequences = tokenizer.texts_to_sequences(news)
    padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

    # Make predictions
    predictions = model.predict(padded_sequences)

    # Decode predictions
    categories = ['business', 'entertaintment', 'politics', 'sport', 'tech']  # Define your category names here
    predicted_categories = [categories[np.argmax(pred)] for pred in predictions]

    return f"{predictions} \n {predicted_categories}"

In [9]:
# Example news articles
# sports, entertaintment, business, chunab, tiktok, politics, tech, tech
news_list = [
    "काठमाडौँ — नेपाली राष्ट्रिय क्रिकेट टोलीका कप्तान रोहितकुमार पौडेलले ट्वान्टी–२० विश्वकपअगाडि सबै खेलाडीबारे जानकारी लिन पनि वेस्ट इन्डिज ‘ए’ सँग बुधबारको खेलमा केही खेलाडीलाई अवसर दिएको बताएका छन् ।  बुधबारको तेस्रो खेलमा नेपालले नियमित कप्तान रोहितसँगै सोमपाल कामी र आसिफ सेखलाई प्लेइङ ११ मा समावेश गरेको थिएन ।",
    "तमिल सुपरस्टार रजनीकान्त पाँच दशकदेखि फिल्मी क्षेत्रमा सक्रिय छन् । उमेरले सात दशक पार गरिसकेका रजनीकान्त आगामी फिल्म ‘कुली’ मा भने सुन तस्करसँग भिडिरहेका देखिनेछन् । लोकेश कनगराजको निर्देशनमा बन्ने ‘कुली’ उनको १७१ औं फिल्म हो । काठमाडौँ — तमिल सुपरस्टार रजनीकान्त जति चर्चित छन्, त्यति नै सरल र उदार । संसारभर करोडौं फ्यान छन् । उनी उमेरले सात दशक नाघिसके । तर फिल्मी क्षेत्रमा अझै पनि निकै सक्रिय । नायक मात्र होइन, चरित्र अभिनेता होस् वा खलनायकको भूमिकामा निकै जम्छन् उनी । पर्दामा भयंकर एक्सन गरिरहेका उनी वास्तवमा भने निकै सरल रजनीकान्तको कुरै बेग्लै । उनको पछिल्लो पिःम याद छ ? त्यसमा उनी जेलर थिए । ",
    "सन् २०२४ का लागि विश्व बैंकले ३.३ र एडीबीले ३.६ प्रतिशत आर्थिक वृद्धि हुने प्रक्षेपण गरिसकेका छन् । चालु आर्थिक वर्षमा सरकारले ६ प्रतिशत आर्थिक वृद्धिको लक्ष्य तय गरेको थियो । गत आर्थिक वर्ष आधारभूत मूल्यमा २.३१ प्रतिशत आर्थिक वृद्धिदर रहेको संशोधित कार्यालयको अनुमान छ । चालु आर्थिक वर्षको ९ महिना (गत चैत) सम्मको वास्तविक तथ्यांक र आगामी तीन महिना (वैशाखदेखि असारसम्म) को अनुमानित तथ्यांकका आधारमा कार्यालयले राष्ट्रिय लेखा तथ्यांक सार्वजनिक गरेको हो । उपभोक्ता मूल्यमा भने यस वर्ष नेपालको आर्थिक वृद्धिदर ३.८७ प्रतिशत हुने देखिएको छ । उपभोक्ताको मूल्यमा गत आर्थिक वर्ष ०७९/८० मा आर्थिक वृद्धिदर १.९५ प्रतिशत हुने कार्यालयको संशोधित अनुमान छ । आधारभूत मूल्यमा कर र अनुदान समावेश गरेपछि उपभोक्ता मूल्य आउँछ । यसकारण आधारभूत मूल्यको तुलनामा उपभोक्ता मूल्यमा गणना गरिएको आर्थिक वृद्धिदर बढी हुने गर्छ । तर गत वर्ष लक्ष्यको तुलनामा राजस्व संकलन झन्डै १८ प्रतिशतले घटेकाले आधारभूत मूल्यमा गणना गरिएको आर्थिक वृद्धिदर उपभोक्ता मूल्यमा गणना गरिएको भन्दा बढी देखिएको थियो । तर यस वर्षदेखि भने पुरानै अवस्था देखिन्छ ।",
    "भारतमा अहिले निर्वाचनको मौसम छ। कुल सात चरणमा हुने निर्वाचनको दुई चरण सकिइसकेको छ। जम्माजम्मी गरी ४७ दिनमा सम्पन्न हुने निर्वाचनमा १ अर्ब जनसङ्ख्या हाराहारी भारतीयहरूले भाग लिनेछन्। संसारको सबैभन्दा ठूलो लोकतन्त्र मानिने भारतमा ४०० भन्दा धेरै भाषा बोलिन्छ। विभिन्न समुदाय, धर्म, संस्कृति र परम्परा भएको भारत आफैंमा एउटा उपमहाद्वीप सरह छ। उत्तर र दक्षिणको भाषिक भिन्नता मात्र नभएर, एकै हिन्दु धर्ममा पनि फरक फरक विश्वास र परम्परा रहेका छन्। त्यस्तै मुस्लिम, जैन, फारसी, बौद्ध, क्रिश्चियनदेखि यहुदीसम्म बसोबास गरिरहेकोले भारत एक ठूलो विविधता बोकेको देश हो। भौगोलिक विविधता त यसै पनि हुने नै भयो। विश्वको सबैभन्दा धेरै जनसङ्ख्या भएको संसारको पाँचौं ठूलो अर्थतन्त्र तथा इतिहासमा विभाजनको घाउ बोकेर पनि आर्थिक विकास र लोकतान्त्रिक मूल्यमान्यतामा अडिग देश भनेर विश्वभर भारतलाई मानिने गरिएको छ।",
    "देशका इन्टरनेट कम्पनीसम्बन्धी नीति-नियमहरूमा कडाइ गर्ने चिनियाँ सरकारका नयाँ प्रावधानहरू बुधवारबाट लागु भएका छन्। ती नयाँ नियमहरू कसरी कार्यान्वयन हुन्छन् भन्ने चिन्ताहरू पनि व्यक्त भइरहेका छन्। 'स्टेट सेक्रेट ल' भनिने राज्यका सूचनहरू गोप्य राख्नुपर्ने कानुनको विस्तारित नयाँ नियम विभिन्न कम्पनीहरू तथा टेन्सेन्ट, बाइटड्यान्स र विबोजस्ता सामाजिक सञ्जाल सञ्जालकहरूले लागु गर्न बाध्य हुने छन्।",
    "कँडेलले नेकपा (एमाले)को भागमा परेको कानून मन्त्रालय आफैंसँग राखेर मन्त्रिपरिषद् विस्तार गरेका छन्। उनले एमालेबाट भौतिक पूर्वाधार तथा शहरी विकासमन्त्रीमा शेरबहादुर बुढा र भूमि, व्यवस्था, कृषि तथा सहकारीमन्त्रीमा विनोदकुमार शाहलाई नियुक्त गरेका छन्। यस्तै, गोमता विक भौतिक राज्यमन्त्री नियुक्त भएकी छन्। यसैगरी, नेकपा (माओवादी केन्द्र)बाट आर्थिक मामिला तथा योजनामन्त्रीमा महेन्द्र केसी, ऊर्जा तथा जलस्रोतमन्त्रीमा रणसिंह परियार, सामाजिक विकासमन्त्रीमा वीरबहादुर शाही र दुर्ग रावत उद्योग, पर्यटन, वन तथा वातावरणमन्त्री बनेका छन्। यीमध्ये नवनियुक्त मन्त्रीले पद तथा गोपनीयताको शपथ लिए पनि कृषिमन्त्री नियुक्त भएका शाह भने शपथ ग्रहणमा उपस्थित भएनन्।",
    "काठमाडौं । राष्ट्रिय सूचना तथा सञ्चार प्रविधि दिवसको अवसरमा सरकारले नेसनल आईसीटी अवार्ड २०२४ वितरण गरिएको हो। तर यस पटक सूचना प्रविधि क्षेत्रमा योगदाननै नगरेका र सञ्चारमन्त्री रेखा शर्माका नजिकका व्यक्तिलाई नेसनल आइसिटी एक्सिलेन्स अवार्ड २०२४ दिईएको भन्दै विवाद उत्पन्न भएको छ। मन्त्री निकट डा. शैलेन्द्र गिरीलाई आज आईसिटी डेको अवसरमा आयोजित कार्यक्रममा आईसिटी अवार्ड दिईएको छ। सरकारले आईसिटी अर्वाडका लागि गठन गरेको उपसमितिले सूचना प्रविधि क्षेत्रमा योगदाननै नगरेका व्यक्तिलाई पुरस्कार दिईएको र विवादमा रहेको संस्थालाई समेत अवार्ड दिईएको भन्दै आपत्ति जनाईएको छ।",
    "एटीएण्डटीलाई ५७ मिलियन अमेरिकी डलर, भेरिजोनलाई ४७ मिलयन जरिवाना तिर्न भनिएको छ। यस बिच स्प्रिन्ट र टी–मोबाइललाई कुल ९२ मिलियन अमेरिकी डलर जरिवाना लगाइएको छ। स्प्रिन्ट र टी–मोबाइल दुई अघि मर्ज भएका थिए। जसका कारण उनीहरुको जरिवाना धेरै देखिन्छ। एफसीसीले २०१८ मा उनीहरुका गतिविधिहरू प्रकाशमा आएपछि वाहकहरूको अनाधिकृत खुलासा र ग्राहकहरूको वास्तविक–समय स्थान डाटाको बिक्रीमा गहिरो अनुसन्धान गरेको थियो। एफसीसी आयुक्त जेसिका रोजेनवर्सेलले भनिन्, क्यारियरहरूले “डाटा एग्रीगेटरहरूलाई वास्तविक–समय स्थान जानकारी बेचे, जसमा अत्यधिक संवेदनशील डाटालाई बेल–बन्ड कम्पनीहरू, बाउन्टी हन्टर र अन्य छायादार अभिनेताहरूको हातमा पुग्यो।’ एजेन्सीका अनुसार, सार्वजनिक रिपोर्टहरू पछि यस योजनाको खुलासा हुन सुरु भएको थियो। मिसौरीका एक शेरिफले सेक्युरस नामको कम्पनीले वायरलेस क्यारियरहरूबाट प्राप्त गरेको स्थान जानकारी प्रयोग गरेर धेरै व्यक्तिहरूलाई ट्र्याक गरिरहेको थियो। सेक्युरसले देशमा सुधारात्मक सुविधाहरूमा सञ्चार सेवाहरू प्रदान गर्ने इनग्याजेडले उल्लेख गरेको छ।",
    "चौथो मारवाडी क्रिकेट लिग हुने कान्तिपुर संवाददाता  नेपाल राष्ट्रिय मारवाडी युवा संगठनको आयोजनामा वैशाख २२ देखि विभिन्न सहरमा चौथो संस्करणको नेपाल मारवाडी क्रिकेट लिग प्रतियोगिता हुने भएको छ ।  ",
    "सूचना प्रविधि दिवसमै इन्टरनेट डाउन कान्तिपुर संवाददाता  बिहीबार विभिन्न कार्यक्रम गरेर सञ्चार तथा सूचना प्रविधि दिवस मनाइरहँदा देशभर अधिकांश इन्टरनेट सेवा भने अवरुद्ध हुन पुगेका छन् । वर्ल्डलिङ्क, सुबिसु लगायतका प्रमुख इन्टरनेट सेवा प्रदायकको नेटवर्क स्लो भएको छ भने भायोनेटको नेटवर्क शतप्रतिशत डाउन भएको छ । ",
    "नेप्से परिसूचक र कारोबार रकम दुबै घट्यो कान्तिपुर संवाददाता  साप्ताहिक कारोबारको अन्तिम दिन बिहीबार नेप्से परिसूचक ७.३१ अंकले घटेर १ हजार ९ सय ९८.९६ अंकमा बन्द भएको छ । "
]

In [10]:
#load saved models
model_simple_rnn = load_model('/content/drive/MyDrive/Colab Notebooks/nlp/model_rnn.keras')
model_lstm = load_model('/content/drive/MyDrive/Colab Notebooks/nlp/model_lstm.keras')
model_gru = load_model('/content/drive/MyDrive/Colab Notebooks/nlp/model_gru.keras')

In [11]:
max_len = 500  # Max length of sequence
vocab_size = 50000  # Vocabulary size
embedding_dim = 350

In [14]:
with open('/content/drive/MyDrive/Colab Notebooks/nlp/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [15]:
# Predict categories using Simple RNN Model
predicted_categories_simple_rnn = predict_category(model_simple_rnn, tokenizer, max_len, news_list)
print("Predicted Categories (Simple RNN Model):", predicted_categories_simple_rnn)


1/1 [==============================] - 0s 315ms/step
Predicted Categories (Simple RNN Model): [[9.81389638e-03 1.84486046e-01 1.21033855e-03 2.86269009e-01
  5.18220723e-01]
 [8.29517841e-01 1.00312233e-02 1.28942095e-02 1.37505099e-01
  1.00517068e-02]
 [9.70551610e-01 5.02522988e-03 1.90955717e-02 2.30991311e-04
  5.09652914e-03]
 [3.09904255e-02 6.89194538e-03 3.21427186e-04 6.74126903e-03
  9.55054998e-01]
 [4.29999769e-01 5.66014498e-02 4.41935897e-01 5.72994761e-02
  1.41633367e-02]
 [1.42556680e-02 2.48270750e-01 2.90849851e-03 1.43037364e-01
  5.91527700e-01]
 [7.29450490e-03 7.99628124e-02 8.93589318e-01 1.86087266e-02
  5.44667360e-04]
 [9.83850565e-03 1.28992185e-01 1.74166239e-03 4.42642858e-03
  8.55001152e-01]
 [3.98849283e-04 1.72769446e-02 1.20069901e-03 9.53806520e-01
  2.73171067e-02]
 [2.33952748e-03 1.08852773e-03 1.47656025e-03 9.93310690e-01
  1.78468390e-03]
 [1.66387595e-02 9.42870080e-02 6.34974800e-04 3.89182679e-02
  8.49520922e-01]] 
 ['tech', 'business', 'b

In [16]:
# Predict categories using LSTM Model
predicted_categories_lstm = predict_category(model_lstm, tokenizer, max_len, news_list)
print("Predicted Categories (LSTM Model):", predicted_categories_lstm)

1/1 [==============================] - 0s 438ms/step
Predicted Categories (LSTM Model): [[1.5425469e-05 2.9188988e-03 2.5456937e-04 9.9673611e-01 7.4964628e-05]
 [7.8690318e-06 9.9903852e-01 3.7598002e-04 4.2121802e-04 1.5634559e-04]
 [9.9937195e-01 1.2662107e-06 2.1482412e-04 2.6858174e-07 4.1172994e-04]
 [4.7889628e-04 9.7650969e-01 1.9501689e-03 1.4863058e-03 1.9574916e-02]
 [3.0130942e-03 4.7474258e-02 5.1473489e-04 5.8586197e-04 9.4841206e-01]
 [1.3520345e-02 1.4426805e-01 4.7264933e-03 2.4733758e-02 8.1275129e-01]
 [2.2023189e-04 9.8717052e-01 6.5093194e-03 1.0561120e-03 5.0438396e-03]
 [1.6105713e-04 9.5302326e-05 1.5833033e-06 1.5728378e-06 9.9974048e-01]
 [3.2612759e-05 4.9441629e-03 3.9452151e-04 9.9447614e-01 1.5255141e-04]
 [9.8869135e-04 1.6590460e-04 6.2656777e-06 8.5421661e-06 9.9883062e-01]
 [9.9910897e-01 2.3019718e-06 4.2860876e-04 8.1202728e-07 4.5929474e-04]] 
 ['sport', 'entertaintment', 'business', 'entertaintment', 'tech', 'tech', 'entertaintment', 'tech', 'sport

In [17]:
# Predict categories using GRU Model
predicted_categories_gru = predict_category(model_gru, tokenizer, max_len, news_list)
print("Predicted Categories (GRU Model):", predicted_categories_gru)

1/1 [==============================] - 1s 628ms/step
Predicted Categories (GRU Model): [[6.17579892e-02 5.17535070e-03 1.80007312e-02 9.14132476e-01
  9.33516130e-04]
 [4.92977015e-05 9.97915506e-01 7.73256412e-04 1.16393011e-07
  1.26192311e-03]
 [9.99555528e-01 2.26650241e-06 1.27739286e-08 1.55162415e-04
  2.86994647e-04]
 [2.37934734e-03 3.80683783e-03 3.39531312e-07 4.08880716e-08
  9.93813455e-01]
 [3.06768040e-03 3.09057921e-01 1.00744197e-04 6.74200692e-07
  6.87772989e-01]
 [2.00462155e-02 2.37779453e-01 7.09678888e-01 1.67383216e-02
  1.57570578e-02]
 [9.57904034e-04 9.25757527e-01 3.31662316e-03 7.45436353e-07
  6.99671954e-02]
 [7.30660046e-04 1.04144774e-01 1.24064682e-05 6.23153724e-08
  8.95112038e-01]
 [2.58322917e-02 1.76430936e-03 6.32388936e-03 9.65766609e-01
  3.12929857e-04]
 [5.09038614e-03 9.77252028e-04 1.09619052e-07 5.39889946e-08
  9.93932188e-01]
 [9.75641668e-01 1.48947784e-05 1.21816640e-06 2.42052563e-02
  1.37006020e-04]] 
 ['sport', 'entertaintment', 'b

**Make an API for news prediction.**

In [21]:
!pip install fastapi uvicorn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
from fastapi import FastAPI, Body
import uvicorn
from pyngrok import ngrok
from typing import List

In [26]:
app = FastAPI()


@app.get('/')
def hello():
    return {'message': 'Hello, Welcome to News Classifier.'}

# news prediction function
@app.post('/predict/')
def predict_news(news_list: List[str]):
    pred_rnn = predict_category(model_simple_rnn, tokenizer, max_len, news_list)
    pred_lstm = predict_category(model_lstm, tokenizer, max_len, news_list)
    pred_gru = predict_category(model_gru, tokenizer, max_len, news_list)

    return {
        "RNN Prediction: ": pred_rnn,
        "LSTM Prediction: ": pred_lstm,
        "GRU Prediction: ": pred_gru
    }


In [27]:
# Authenticate ngrok with your token
ngrok.set_auth_token("2fwQcKQqJFepl8UkcpqyK0lsDl2_6BzJGMbjHNAW79PyfK3VP")

In [28]:
# Run FastAPI app using Uvicorn with Ngrok
import nest_asyncio
import uvicorn

# Run FastAPI app
def run_app():
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start Ngrok tunnel
def start_ngrok():
    ngrok_tunnel = ngrok.connect(8000)
    print("Public URL:", ngrok_tunnel.public_url)
    return ngrok_tunnel

if __name__ == "__main__":
    tunnel = start_ngrok()
    run_app()

Public URL: https://c17a-34-16-206-135.ngrok-free.app


INFO:     Started server process [288]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.10.28.213:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.10.28.213:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.10.28.213:0 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 [==============================] - 0s 43ms/step
INFO:     103.10.28.213:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 [==============================] - 0s 45ms/step
INFO:     103.10.28.213:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 [==============================] - 0s 47ms/step
INFO:     103.10.28.213:0 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [288]


KeyboardInterrupt: 

In [ ]:
ngrok.kill()